In [35]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


machine = pd.read_csv(r"C:\Users\CalumBrown\OneDrive - Blend 360\Documents\Personal Development\Machine Learning Interview\Cleaned Machine Failure Dataset.csv", index_col=0)

In [36]:
x = machine[['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']]
# Feature names cant contain symbols for xgboost model
x = x.rename(columns = {'Air temperature [K]':'Air temperature', 'Process temperature [K]':'Process temperature','Rotational speed [rpm]': 'Rotational speed', 'Torque [Nm]':'Torque', 'Tool wear [min]': 'Tool wear'})
y = machine['Failure Type']
y2 = machine['Failure']

In [37]:
# XGBoost model on failure type

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size = 0.3, random_state=42)


In [38]:
# Encoding failure type column after split to avoid data leakage
# Encoding our target variable
# Should do this after split to avoid data leakage
Label_Encoder = LabelEncoder()
ytrain = Label_Encoder.fit_transform(ytrain)
ytest = Label_Encoder.fit_transform(ytest)


In [39]:
#Fitting model 
xgb1 = xgb.XGBClassifier(random_state = 42)
xgb1.fit(xtrain, ytrain)
xgb1_predict = xgb1.predict(xtest)


In [40]:
## Checking accuracy score and metrics

print(accuracy_score(ytest, xgb1_predict))
print(confusion_matrix(ytest, xgb1_predict))
print(classification_report(ytest, xgb1_predict))

0.9819789187351241
[[  29    3    0    1    0    0]
 [   0 2831    4    6    0    3]
 [   0    6   13    2    0    0]
 [   1   13    0   15    0    0]
 [   0    5    0    0    0    0]
 [   0    9    0    0    0    0]]
              precision    recall  f1-score   support

           0       0.97      0.88      0.92        33
           1       0.99      1.00      0.99      2844
           2       0.76      0.62      0.68        21
           3       0.62      0.52      0.57        29
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         9

    accuracy                           0.98      2941
   macro avg       0.56      0.50      0.53      2941
weighted avg       0.98      0.98      0.98      2941



c:\Users\CalumBrown\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\CalumBrown\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\CalumBrown\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
# Hyperparameter tuning 
# It is our job to set parameters which can influence the model fit
# These are different paramaters and theory is quite complex but all impact how model works
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.85, 1.0],
    'colsample_bytree': [0.7, 0.85, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1, 10, 100],
    'reg_lambda': [0.5, 0.7, 1, 1.3]
}

# We then use cross validation to test different combinations, loop through each one of these validation folds
# Can either loop through every combination (gridcv) or random (randomcv)
#This uses its own cross validation system so no need for validation split as just lose important training data


In [42]:
# fitting model testing different params 

xgb_model = xgb.XGBClassifier(random_state=42)

grid_search = RandomizedSearchCV(xgb_model, param_grid, cv=10, scoring="accuracy", n_iter=100, n_jobs=-1, verbose=2, random_state=42)
grid_search.fit(xtrain, ytrain)

#Returns model with best score - is this what you always want?
best_xgb = grid_search.best_estimator_

# Best parameters from tuning
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

xgbparams_predict = best_xgb.predict(xtest)

print(accuracy_score(ytest, xgbparams_predict))
print(confusion_matrix(ytest, xgbparams_predict))
print(classification_report(ytest, xgbparams_predict))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best Parameters: {'subsample': 1.0, 'reg_lambda': 0.5, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.85}
Best Accuracy: 0.9785714285714284
0.9816388983339001
[[  30    2    0    1    0    0]
 [   1 2835    3    5    0    0]
 [   0    8   11    2    0    0]
 [   1   16    1   11    0    0]
 [   0    5    0    0    0    0]
 [   0    9    0    0    0    0]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.92        33
           1       0.99      1.00      0.99      2844
           2       0.73      0.52      0.61        21
           3       0.58      0.38      0.46        29
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         9

    accuracy                           0.98      2941
   macro avg       0.54      0.47      0.50      2941
weighted avg    

c:\Users\CalumBrown\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\CalumBrown\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\CalumBrown\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [43]:
### Testing binary machine failure with xgboost

x2train, x2test, y2train, y2test = train_test_split(x,y2,test_size = 0.3, random_state=42)


In [44]:
#Fitting model with no fine tuning
xgbfail = xgb.XGBClassifier(random_state = 42)
xgbfail.fit(x2train, y2train)
xgbfail_predict = xgbfail.predict(x2test)

print(accuracy_score(y2test, xgbfail_predict))
print(confusion_matrix(y2test, xgbfail_predict))
print(classification_report(y2test, xgbfail_predict))

0.9802788167290037
[[2825   19]
 [  39   58]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2844
           1       0.75      0.60      0.67        97

    accuracy                           0.98      2941
   macro avg       0.87      0.80      0.83      2941
weighted avg       0.98      0.98      0.98      2941



In [45]:
# Hyper parameter tuning xgboost model
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.85, 1.0],
    'colsample_bytree': [0.7, 0.85, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1, 10, 100],
    'reg_lambda': [0.5, 0.7, 1, 1.3]
}

xgb_modelfail = xgb.XGBClassifier(random_state=42)

grid_search = RandomizedSearchCV(xgb_modelfail, param_grid, cv=10, scoring="accuracy", n_iter=100, n_jobs=-1, verbose=2, random_state=42)
grid_search.fit(x2train, y2train)

#Returns model with best score - is this what you always want?
best_xgbfail = grid_search.best_estimator_

# Best parameters from tuning
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

xgbparamsfail_predict = best_xgbfail.predict(x2test)

print(accuracy_score(y2test, xgbparamsfail_predict))
print(confusion_matrix(y2test, xgbparamsfail_predict))
print(classification_report(y2test, xgbparamsfail_predict))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best Parameters: {'subsample': 1.0, 'reg_lambda': 0.5, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Best Accuracy: 0.9774052478134113
0.9816388983339001
[[2827   17]
 [  37   60]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2844
           1       0.78      0.62      0.69        97

    accuracy                           0.98      2941
   macro avg       0.88      0.81      0.84      2941
weighted avg       0.98      0.98      0.98      2941



In [46]:
# Trying with recall_macro as evaluation optimal score

xgb_modelfail = xgb.XGBClassifier(random_state=42)

grid_search = RandomizedSearchCV(xgb_modelfail, param_grid, cv=10, scoring="recall_macro", n_iter=100, n_jobs=-1, verbose=2, random_state=42)
grid_search.fit(x2train, y2train)

#Returns model with best score - is this what you always want?
best_xgbfail = grid_search.best_estimator_

# Best parameters from tuning
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

xgbparamsfail_predict = best_xgbfail.predict(xtest)

print(accuracy_score(y2test, xgbparamsfail_predict))
print(confusion_matrix(y2test, xgbparamsfail_predict))
print(classification_report(y2test, xgbparamsfail_predict))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best Parameters: {'subsample': 1.0, 'reg_lambda': 0.7, 'reg_alpha': 0.01, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Best Accuracy: 0.7695285710259258
0.9806188371302278
[[2825   19]
 [  38   59]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2844
           1       0.76      0.61      0.67        97

    accuracy                           0.98      2941
   macro avg       0.87      0.80      0.83      2941
weighted avg       0.98      0.98      0.98      2941



In [50]:
# Dropping classification for failure threshold down to 0.4

xgb_modelfail = xgb.XGBClassifier(random_state=42)

grid_search = RandomizedSearchCV(xgb_modelfail, param_grid, cv=10, scoring="accuracy", n_iter=100, n_jobs=-1, verbose=2, random_state=42)
grid_search.fit(x2train, y2train)

#Returns model with best score - is this what you always want?
best_xgbfail = grid_search.best_estimator_

# Best parameters from tuning
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

probs = best_xgbfail.predict_proba(x2test)[:, 1]  # probability of class 1

# Set a lower threshold, e.g., 0.3 or 0.4 instead of 0.5
threshold = 0.4
rffail1_test = (probs >= threshold).astype(int)


print(accuracy_score(y2test, rffail1_test))
print(confusion_matrix(y2test, rffail1_test))
print(classification_report(y2test, rffail1_test))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best Parameters: {'subsample': 1.0, 'reg_lambda': 0.5, 'reg_alpha': 1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Best Accuracy: 0.9774052478134113
0.981298877932676
[[2821   23]
 [  32   65]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2844
           1       0.74      0.67      0.70        97

    accuracy                           0.98      2941
   macro avg       0.86      0.83      0.85      2941
weighted avg       0.98      0.98      0.98      2941

